**Checklist before running:**

`ALLOWED_PROJECTS`: Make sure this is the Project ID where you will run Notebook 03 (Cloud Run & Load Balancer). If it's the same project, put the same ID.

`MODEL_ID_2`: Ensure you have actually uploaded a model to `us-east4`. If `01-create-clean-model.ipynb` only uploaded to `us-central1`, you must upload it to `us-east4` before running the second half of this cell, or it will fail with 404 Not Found.

In [ ]:
from google.cloud import aiplatform
from google.cloud import aiplatform_v1

In [ ]:
PROJECT_ID = "PLACEHOLDER_YOUR_HOST_PROJECT_ID"
REGION1 = "us-central1"
REGION2 = "us-east4"

PROJECT_NUMBER = "PLACEHOLDER_YOUR_HOST_PROJECT_NUMBER"  # numeric project number
ALLOWED_PROJECTS = ["PLACEHOLDER_YOUR_CLIENT_PROJECT_ID"]
MODEL_ID_1 = "PLACEHOLDER_YOUR_REGION_1_MODEL_ID" 
MODEL_ID_2 = "PLACEHOLDER_YOUR_REGION_2_MODEL_ID" 

In [ ]:
def create_psc_endpoint_and_deploy(display_name, region,model_id, machine_type="n1-standard-4"):
    print(f"\n--- Processing {display_name} ---")

    # 1. Create PSC Endpoint using GAPIC (Low-level Client)
    # We do this because the high-level SDK doesn't fully support PSC config creation yet.
    api_endpoint_client = aiplatform_v1.EndpointServiceClient(
        client_options={"api_endpoint": f"{region}-aiplatform.googleapis.com"}
    )

    endpoint_proto = aiplatform_v1.Endpoint(
        display_name=display_name,
        private_service_connect_config=aiplatform_v1.PrivateServiceConnectConfig(
            enable_private_service_connect=True,
            project_allowlist=ALLOWED_PROJECTS
        )
    )

    print(f"Creating PSC Endpoint (GAPIC): {display_name}...")
    parent = f"projects/{PROJECT_ID}/locations/{region}"
    operation = api_endpoint_client.create_endpoint(parent=parent, endpoint=endpoint_proto)
    response = operation.result() 
    print(f"Endpoint Created: {response.name}")

    # 2. Deploy Model using SDK (High-level Client)
    # We 'adopt' the GAPIC endpoint into the SDK for easier deployment
    endpoint = aiplatform.Endpoint(endpoint_name=response.name)
    
    # Construct full model resource name
    model_resource = f"projects/{PROJECT_NUMBER}/locations/{region}/models/{model_id}"
    model = aiplatform.Model(model_resource)

    print(f"Deploying model {model_id}...")
    endpoint.deploy(
        model=model,
        deployed_model_display_name=f"{display_name}-deployment",
        machine_type=machine_type,
        min_replica_count=1,
        max_replica_count=1
    )
    
    return endpoint

**Deploy the `REGION1` model to an endpoint in `REGION1`**

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION1)

###### endpoint1 = create_psc_endpoint_and_deploy("duplicate-sa-1", MODEL_ID_1)

In [ ]:
endpoint1 = create_psc_endpoint_and_deploy(f"dummy-model-{REGION1}", REGION1, MODEL_ID_1)

**Now deploy the `REGION2` model to an endpoint in `REGION2`**

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION2)

In [ ]:
endpoint2 = create_psc_endpoint_and_deploy(f"dummy-model-{REGION2}", REGION2, MODEL_ID_2)

<mark>**STOP: Take note of your endpoint IDs (needed for load balancer integration)**</mark>

Service attachment
- `projects/n65824f2a7ece296ep-tp/regions/us-central1/serviceAttachments/gkedpm-a08187020203f374cffe318569b3cc`
-  `projects/z8d8be9fe5df7b10b-tp/regions/us-east4/serviceAttachments/gkedpm-81415526f4187fe95b8db2175989b2`

<mark>**STOP: Take note of your Service Attachment paths (needed for load balancer integration)**</mark>